# 사전 학습된 모델 준비

In [1]:
import torch
import torch.nn as nn
from torchsummary import summary

from torchvision.models.vgg import vgg16

device = "cuda" if torch.cuda.is_available() else "cpu"
model = vgg16(pretrained=True) # ❶ vgg16 모델 객체 생성
fc = nn.Sequential( # ❷ 분류층의 정의
       nn.Linear(512 * 7 * 7, 4096),
       nn.ReLU(),
       nn.Dropout(), #❷ 드롭아웃층 정의
       nn.Linear(4096, 4096),
       nn.ReLU(),
       nn.Dropout(),
       nn.Linear(4096, 10,bias=False),
   )
model.classifier = fc # ➍ VGG의 classifier를 덮어씀
model.to(device)
print(model)

/home/user/anaconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/user/anaconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [2]:
summary(model,input_size=(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

# 데이터 전처리와 증강

In [3]:
import tqdm

from torchvision.datasets.cifar import CIFAR10
from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.transforms import RandomHorizontalFlip, RandomCrop, Normalize
from torch.utils.data.dataloader import DataLoader

from torch.optim.adam import Adam

transforms = Compose([
   Resize(224),
   RandomCrop((224, 224), padding=4),
   RandomHorizontalFlip(p=0.5),
   ToTensor(),
   Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.243, 0.261))
])

# 데이터로더 정의

In [4]:
training_data = CIFAR10(root="./", train=True, download=True, transform=transforms)
test_data = CIFAR10(root="./", train=False, download=True, transform=transforms)

train_loader = DataLoader(training_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
"""# 학습 루프 정의"""
for name, param in model.features.named_parameters():
    param.requires_grad = False

params_to_update = []
params_name_to_update = []
for name, param in model.classifier.named_parameters():
    param.requires_grad = True
    params_to_update.append(param)
    params_name_to_update.append(name)

print(params_name_to_update)
print(params_to_update)

['0.weight', '0.bias', '3.weight', '3.bias', '6.weight']
[Parameter containing:
tensor([[ 2.9047e-03, -4.5093e-03,  4.4001e-03,  ..., -3.9697e-03,
          5.2363e-03,  7.3333e-04],
        [ 3.4023e-03, -6.4414e-04,  6.7441e-04,  ...,  8.2047e-05,
         -4.9118e-03,  4.1610e-03],
        [-3.2158e-03,  3.1949e-03,  5.1869e-03,  ..., -4.7055e-03,
         -3.2581e-03, -4.2451e-03],
        ...,
        [-5.1081e-03,  3.0231e-03,  4.1143e-03,  ...,  7.8124e-04,
         -5.0876e-03,  5.6983e-03],
        [-4.6495e-03,  3.9609e-03, -2.0313e-03,  ..., -8.1211e-04,
         -5.5881e-03,  3.8449e-04],
        [-3.9622e-03,  2.8154e-03, -3.9502e-03,  ...,  4.2106e-04,
          5.2583e-03, -3.9908e-03]], requires_grad=True), Parameter containing:
tensor([ 0.0031,  0.0036,  0.0022,  ...,  0.0032, -0.0052,  0.0027],
       requires_grad=True), Parameter containing:
tensor([[ 0.0138,  0.0111,  0.0038,  ...,  0.0150, -0.0115,  0.0145],
        [ 0.0015,  0.0079, -0.0053,  ..., -0.0136,  0.00

# 학습 루프 정의

In [10]:
lr = 1e-4
#optim = Adam(model.parameters(), lr=lr)
optim = Adam(params=params_to_update, lr=lr)
for epoch in range(5):
   iterator = tqdm.tqdm(train_loader) # ➊ 학습 로그 출력
   for data, label in iterator:
       optim.zero_grad()

       preds = model(data.to(device)) # 모델의 예측값 출력

       loss = nn.CrossEntropyLoss()(preds, label.to(device))
       loss.backward()
       optim.step()
     
       # ❷ tqdm이 출력할 문자열
       iterator.set_description(f"epoch:{epoch+1} loss:{loss.item()}")

torch.save(model.state_dict(), "CIFAR_pretrained.pth") # 모델 저장

epoch:1 loss:0.594566822052002:   6%|▌         | 91/1563 [16:46<4:31:28, 11.07s/it] 


KeyboardInterrupt: 

In [ ]:
model.load_state_dict(torch.load("CIFAR_pretrained.pth", map_location=device))

num_corr = 0

with torch.no_grad():
   for data, label in test_loader:

       output = model(data.to(device))
       preds = output.data.max(1)[1]
       corr = preds.eq(label.to(device).data).sum().item()
       num_corr += corr

   print(f"Accuracy:{num_corr/len(test_data)}")

Accuracy:0.929
